In [1]:
from functools import cmp_to_key, lru_cache, total_ordering, partial, partialmethod, reduce, singledispatch, update_wrapper, wraps

## functools.cmp_to_key(func)
----
Transform an old-style comparison function to a key function. Used with tools that accept key functions (such as sorted(), min(), max(), heapq.nlargest(), heapq.nsmallest(), itertools.groupby()). This function is primarily used as a transition tool for programs being converted from Python 2 which supported the use of comparison functions.

A comparison function is any callable that accept two arguments, compares them, and returns a negative number for less-than, zero for equality, or a positive number for greater-than. A key function is a callable that accepts one argument and returns another value to be used as the sort key.

> python2 에서 쓰이던 cmp function을 key function으로 바꾸는 역할을 한다.

In [23]:
def fake_print(some):
    print(some, end = ' ')

In [24]:
def cmp_to_key(mycmp):
    'Convert a cmp= funciton into a key= function'
    class K:
        def __init__(self, obj, *args):
            fake_print(obj)
            self.obj = obj
        def __lt__(self, other):
            fake_print("__lt__")
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            fake_print("__gt__")
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            fake_print("__eq__")
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            fake_print("__le__")
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            fake_print("__ge__")
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            fake_print("__ne__")
            return mycmp(self.obj, other.obj) != 0
    return K

In [25]:
def reverse_numeric(x, y):
    return y - x

def numeric_compare(x, y):
    return x - y

In [27]:
print(sorted([5,2,4,1,3,1], key = cmp_to_key(reverse_numeric))) # 버블 정렬처럼 비교해서 sort 하는거 같음.. 
print(sorted([5,2,4,1,3], key = cmp_to_key(numeric_compare))) # 안 쓸거 같으니깐 패스

5 2 4 1 3 1 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [5, 4, 3, 2, 1, 1]
5 2 4 1 3 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [1, 2, 3, 4, 5]


In [32]:
def custom(x, y):
    return x*x - y*y
print(sorted([5,2,4,1,3], key = cmp_to_key(custom)))

5 2 4 1 3 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [1, 2, 3, 4, 5]


## @functools.lru_cache(maxsize=128, typed=False)
### lru (least recently used)
- - -
Decorator to wrap a function with a memoizing callable that saves up to the maxsize most recent calls. It can save time when an expensive or I/O bound function is periodically called with the same arguments.

Since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

Distinct argument patterns may be considered to be distinct calls with separate cache entries. For example, f(a=1, b=2) and f(b=2, a=1) differ in their keyword argument order and may have two separate cache entries.

If maxsize is set to None, the LRU feature is disabled and the cache can grow without bound. The LRU feature performs best when maxsize is a power-of-two.

If typed is set to true, function arguments of different types will be cached separately. For example, f(3) and f(3.0) will be treated as distinct calls with distinct results.

To help measure the effectiveness of the cache and tune the maxsize parameter, the wrapped function is instrumented with a cache_info() function that returns a named tuple showing hits, misses, maxsize and currsize. In a multi-threaded environment, the hits and misses are approximate.

The decorator also provides a cache_clear() function for clearing or invalidating the cache.

The original underlying function is accessible through the __wrapped__ attribute. This is useful for introspection, for bypassing the cache, or for rewrapping the function with a different cache.

An LRU (least recently used) cache works best when the most recent calls are the best predictors of upcoming calls (for example, the most popular articles on a news server tend to change each day). The cache’s size limit assures that the cache does not grow without bound on long-running processes such as web servers.

In general, the LRU cache should only be used when you want to reuse previously computed values. Accordingly, it doesn’t make sense to cache functions with side-effects, functions that need to create distinct mutable objects on each call, or impure functions such as time() or random().

##### 페이지 교체 알고리즘으로 배웠던 lru 라 비슷하다고 보면 된다. 
##### 그래서 오랜만에 다시 복습
> LRU (Least Recently Used) : 최근에 사용하지 않은 페이지를 교체 <br>
LFU (Least Frequently Used) : 사용 횟수가 가장 적은 페이지를 교체 <br>
NUR (Not Used Recently) : 최근에 사용하지 않은 페이지를 교체 <br>
FIFO(Fisrt In First Out) : 먼저 적재한 페이지부터 교체 <br>
MFU(Most Frequently Used) : 사용 횟수가 가장 많은 페이지를 교체 <br>
OPT(OPTimal replacement) : 가장 오랫동안 사용하지 않을 것으로 예측한 페이지를 교체 (이상적이지만 구현할 수 없음.) <br>
SCR(Second Chance Replacement) : FIFO 기법의 단점을 보완하는 기법으로 교체 대상을 판별하기 전에 참조 비트를 검사하여 1일 때 한 번의 기회를 더 부여
참조 비트가 1이면 큐의 맨 뒤로 피드백.


In [34]:
import urllib
@lru_cache(maxsize=32)
def get_pep(num):
    'Retrieve text of a Python Enghancement Proposal'
    resource = 'http://www.python.org/dev/peps/pep-%04d/' % num
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [35]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep = get_pep(n)
    print(n, len(pep))

8 105879
290 59822
308 57028
320 49607
8 105879
218 46851
320 49607
279 48609
289 50938
320 49607
9991 9


In [37]:
get_pep.cache_info()

CacheInfo(hits=3, misses=8, maxsize=32, currsize=8)

In [59]:
@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [63]:
[fib(n) for n in range(16)] # 와... 역시 cache
# fibonacci 에서는 lfu 가 더 좋지 않을까... 생각만 해봅니다.

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [64]:
print(fib.cache_info())
"""설명
cache hit: 참조하려는 데이터가 캐시에 존재할 때 캐시 히트라 한다.
cache miss: 참조하려는 데이터가 캐시에 존재 하지 않을 때 캐시 미스라 한다.
cache hit ratio : 적중률 = (캐시히트횟수) / (전체참조횟수)
"""

CacheInfo(hits=128, misses=50, maxsize=None, currsize=50)


'설명\ncache hit: 참조하려는 데이터가 캐시에 존재할 때 캐시 히트라 한다.\ncache miss: 참조하려는 데이터가 캐시에 존재 하지 않을 때 캐시 미스라 한다.\ncache hit ratio : 적중률 = (캐시히트횟수) / (전체참조횟수)\n'